In [1]:
# Imports

from astropy import units as u
import numpy as np
from math import pi

from poliastro.bodies import Earth, Mars, Sun
from poliastro.twobody import Orbit

# Needed to handle TLE into Poliastro's Orbit
from tletools import TLE

# Needed for defining manouvers
from poliastro.maneuver import Maneuver
from poliastro.twobody import thrust
from poliastro.twobody.propagation import cowell

from perylune.orbit_tools import *

In [2]:
# Create circular, equatorial orbit:

orb1 = Orbit.circular(Earth, 300 * u.km, 0*u.deg)
orb2 = Orbit.circular(Earth, 300 * u.km, 60*u.deg)

# Calculate the delta-v cost of inclination change from 87 to 88.4 degrees
dv_inc = inc_change(orb1, orb2)
print("Inclination change from %s to %s requires delta-v %s" % (orb1.inc, orb2.inc, dv_inc))

Inclination change from 0.0 deg to 60.0 deg requires delta-v 7.725760463451862 km / s


In [7]:
# Now calculate the cost for 60 deg change for eccentric orbits (300km)
alt = 300*u.km
orb3 = Orbit.from_classical(Earth, Earth.R + alt, 0.1*u.one, 0.0*u.deg, 0*u.deg, 0*u.deg, 0*u.deg)
orb4 = Orbit.from_classical(Earth, Earth.R + alt, 0.1*u.one, 60.0*u.deg, 0*u.deg, 0*u.deg, 0*u.deg)
orb3_orig = orb3
print_orb(orb3)

6010 x 7346 km x 0.0 deg (GCRS) orbit around Earth (♁) at epoch J2000.000 (TT)
𝑎=6678.14m, b=6644.66m, e=0.10, i=0.00deg raan(Ω)=0.00deg argp(𝜔)=0.00deg nu(𝜈)=0.00deg period=5431.176641180842 s perapis=6010.322939999999 m apoapsis=7345.95026 m


In [4]:
# Calculate velocity vector magnitudes for 100 intervals, each 5 seconds
samples = 200
delta_t = orb3.period / (samples - 1)
vel = calc_vel_inc_cost(orb3, orb4, delta_t, samples)
for v,t,nu,argp, dv in vel:
    nu2pi = nu + 2*pi*u.rad*(nu<0) # nu normalized to 0..2pi
    time_to_periapis = argp - nu2pi
    #print("v=%4.2f %s at %s, nu=%s time to argp=%s, delta-v cost=%s" % (v.value, v.unit, t.iso, nu, (time_to_periapis), abs(dv)))

In [5]:
import plotly.express as px
import plotly.graph_objs as go
import numpy as np

# y - blue - orbital velocity [km/s]
# y2 - red - delta-v cost for inclination change [km/s]

# Note the lowest cost is when the object passes lines of apides (i.e. in the middle between apoapsis and periapsis)

nu_tbl = []
v_tbl = []
dv_tbl = []
for v,t,nu,argp, dv in vel:
    nu_tbl.append(nu.value)
    v_tbl.append(v.value)
    dv_tbl.append(abs(dv.value))

fig = go.Figure()
fig.add_trace(go.Scatter(x = nu_tbl, y = dv_tbl, mode="markers", name="Delta-v [km/s]"))
fig.add_trace(go.Scatter(x = nu_tbl, y = v_tbl, mode="markers", name="Orbital velocity [km/s]"))

# Update legend to show at the top
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
), xaxis_title="True anomaly 𝜈 [rad]", yaxis_title="Veloicty [km/s]", title="Orbit 300x300km", margin=dict(l=20, r=20, t=20, b=20))

fig.show()


In [24]:
interval = 10*u.min
samples = 200
delta_t = interval / samples
orb3 = orb3_orig
print("BEFORE nu=%s" % repr(orb3.nu))
orb3 = orb3.propagate(900*u.s)
print("AFTER nu=%s" % repr(orb3.nu))
vel = calc_vel_inc_cost(orb3, orb4, delta_t, samples)

nu_tbl = []
v_tbl = []
dv_tbl = []
for v,t,nu,argp, dv in vel:
    nu_tbl.append(nu.value)
    v_tbl.append(v.value)
    dv_tbl.append(abs(dv.value))

fig = go.Figure()
fig.add_trace(go.Scatter(x = nu_tbl, y = dv_tbl, mode="markers", name="Delta-v [km/s]"))

# Update legend to show at the top
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
), xaxis_title="True anomaly 𝜈 [rad]", yaxis_title="Veloicty [km/s]", title="Orbit 300x300km", margin=dict(l=20, r=20, t=20, b=20))

fig.show()


BEFORE nu=<Quantity 0. deg>
AFTER nu=<Quantity 1.22435038 rad>


In [6]:
# This is how the orbit can be propagated (the object moving forward in time)
# Observing how the velocity and position vectors change over time
print("orb1 v=%s, r=%s" % (orb1.v, orb1.r))
orb1 = orb1.propagate(1 * u.min) # Let's see how far it moved in 1 minute
print("orb3 v=%s, r=%s" % (orb1.v, orb1.r))

orb1 v=[0.         7.72576046 0.        ] km / s, r=[6678.1366    0.        0.    ] km
orb3 v=[-0.53583321  7.70715625  0.        ] km / s, r=[6662.05514628  463.17348383    0.        ] km
